In [16]:
from typing import List
import json
from datetime import datetime
from copy import deepcopy

In [119]:
def read_jsonl(filepath: str) -> List[dict]:
    with open(filepath) as f:
        return [json.loads(line) for line in f]

def write_jsonl(data: List[dict], filepath: str):
    with open(filepath, "w") as f:
        for line in data:
            f.write(json.dumps(line, ensure_ascii=False) + "\n")

In [132]:
logs_data = read_jsonl("/Users/vinhnguyen/Projects/ext-chatbot/resources/logs/exports_1766646264633-lf-traces-export-cmio1vjfs000ynv0795mpsunw.jsonl")

In [133]:

group_by_sessionId = {}
for s in logs_data:
    sessionId = s["sessionId"]
    if sessionId not in group_by_sessionId:
        group_by_sessionId[sessionId] = []
    if "guru" in s["tags"]:
        group_by_sessionId[sessionId].append(s)

for sessionId in group_by_sessionId:
    sorted_list = sorted(group_by_sessionId[sessionId], key=lambda x: datetime.fromisoformat(x['timestamp'].replace('Z', '+00:00')))
    group_by_sessionId[sessionId] = sorted_list

print(len(group_by_sessionId))


23


In [134]:
conversations = []
for sessionId in list(group_by_sessionId.keys()):
    conversation = []
    if len(group_by_sessionId[sessionId]) == 0:
        continue
    current_ind = len(group_by_sessionId[sessionId])-1
    count = 0
    while True:
        current_history = json.loads(group_by_sessionId[sessionId][current_ind]["metadata"]["history"])
        conversation = current_history + conversation
        current_ind -= len(current_history)//2
        if current_ind <= 0:
            break
        count += 1
        if count > 1000:
            print(current_ind)
            break
    
    try:
        conversation.extend([
            {"role": "user", "content": group_by_sessionId[sessionId][-1]["input"]},
            {"role": "assistant", "content": json.loads(group_by_sessionId[sessionId][-1]["output"])["user_msg"]}
        ])
    except Exception:
        conversation.extend([
            {"role": "user", "content": group_by_sessionId[sessionId][-1]["input"]},
        ])
    refined_conversation = []
    for ind in range(len(conversation)):
        if ind == 0:
            refined_conversation.append(conversation[ind])
            continue
        if conversation[ind]["role"] == conversation[ind-1]["role"]:
            if conversation[ind]["content"] == conversation[ind-1]["content"]:
                continue
            else:
                new_content = conversation[ind-1]["content"] + conversation[ind]["content"]
                refined_conversation.pop()
                refined_conversation.append({
                    "role": conversation[ind]["role"], 
                    "content": new_content
                })
        else:
            refined_conversation.append(conversation[ind])
    conversations.append(refined_conversation)

1
1


In [135]:
len(conversations)

23

In [136]:
len(group_by_sessionId)

23

In [137]:
test_dataset = []
for conversation in conversations:
    for ind in range(len(conversation)):
        if conversation[ind]["role"] == "user":
            test_dataset.append({
                "conversation_history": conversation[:ind],
                "current_message": conversation[ind]["content"]
            })

print(len(test_dataset))

181


In [138]:
import random

write_jsonl(
    random.sample(test_dataset, min(200, len(test_dataset))), 
    "/Users/vinhnguyen/Projects/ext-chatbot/resources/logs/batdongsan_test.jsonl"
)